In [1]:
import torch
import torchvision

from torchvision import models

import datasets.deep_fashion_ctsrbm

import os
import pathlib

---

In [2]:
model = torchvision.models.efficientnet_v2_s()

In [3]:
with open("misc/efficientnet_v2_model_arch.txt", "w") as out_file:
    print(model, file=out_file)

---

In [4]:
model = torchvision.models.efficientnet_v2_s()

In [5]:
ctsrbm_image_transform = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = datasets.deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform)

In [6]:
ctsrbm_image_transform

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [7]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.features.register_forward_hook(get_activation('features'))
model.avgpool.register_forward_hook(get_activation('avgpool'))
model.classifier.register_forward_hook(get_activation('classifier'))

In [8]:
input = ctsrbm_dataset[0][0][None, :]
output = model(input)

In [9]:
print("{:20s} {:}".format("input", input.shape))
print("{:20s} {:}".format("output", output.shape))

print("---")

for key, item in activation.items():
    print("{:20s} {:}".format(key, item.shape))

input                torch.Size([1, 3, 384, 384])
output               torch.Size([1, 1000])
---
features             torch.Size([1, 1280, 12, 12])
avgpool              torch.Size([1, 1280, 1, 1])
classifier           torch.Size([1, 1000])


---

In [10]:
model = torchvision.models.efficientnet_v2_s()

In [11]:
ctsrbm_image_transform = torchvision.models.EfficientNet_B2_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True
ctsrbm_image_transform.crop_size = [768]
ctsrbm_image_transform.resize_size = [768]
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = datasets.deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform)

In [12]:
ctsrbm_image_transform

ImageClassification(
    crop_size=[768]
    resize_size=[768]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [13]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.features.register_forward_hook(get_activation('features'))
model.avgpool.register_forward_hook(get_activation('avgpool'))
model.classifier.register_forward_hook(get_activation('classifier'))

In [14]:
input = ctsrbm_dataset[0][0][None, :]
output = model(input)

In [15]:
print("{:20s} {:}".format("input", input.shape))
print("{:20s} {:}".format("output", output.shape))

print("---")

for key, item in activation.items():
    print("{:20s} {:}".format(key, item.shape))

input                torch.Size([1, 3, 768, 768])
output               torch.Size([1, 1000])
---
features             torch.Size([1, 1280, 24, 24])
avgpool              torch.Size([1, 1280, 1, 1])
classifier           torch.Size([1, 1000])


---

In [16]:
model = torchvision.models.efficientnet_v2_s()

In [17]:
ctsrbm_image_transform = torchvision.models.EfficientNet_B2_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True
ctsrbm_image_transform.crop_size = [768]
ctsrbm_image_transform.resize_size = [768]
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = datasets.deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform)

In [18]:
ctsrbm_image_transform

ImageClassification(
    crop_size=[768]
    resize_size=[768]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [19]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.features.register_forward_hook(get_activation('features'))

In [26]:
last_perm_size = 0

for size in range(33, 768 + 1):

    input = ctsrbm_dataset[0][0][None, :, :size, :size]
    output = model(input)

    for key, item in activation.items():
        if key == "features":
            perm_size = item.size(3)

    if last_perm_size < perm_size:
        print(size, perm_size)
        last_perm_size = perm_size

33 2
65 3
97 4
129 5
161 6
193 7
225 8
257 9
289 10
321 11
353 12
385 13
417 14
449 15
481 16
513 17
545 18
577 19
609 20
641 21
673 22
705 23
737 24


In [27]:
for size in range(33, 768 + 1):

    input = ctsrbm_dataset[0][0][None, :, :size, :size]
    output = model(input)

    for key, item in activation.items():
        if key == "features":
            perm_size = item.size(3)

    theo_perm_size = ((size - 1) // 32) + 1

    if perm_size != theo_perm_size:
        print(size, perm_size, theo_perm_size)